In [1]:
import os
import sys; sys.path.append('/Users/dlchang/img_pipe/')
import img_pipe
reload(img_pipe)
import numpy as np
import scipy
%matplotlib inline

# Img_pipe Recon Demo

This tutorial will walk through how to run a recon using the img_pipe package. This includes MRI alignment, CT registration, electrode localization, anatomical labeling, and nonlinear warping. 

### Set-up

In your SUBJECTS_DIR (where you want to place your subjects imaging data), you should make a new directory (with your subject name, i.e. 'EC86'). Then, create subdirectories __*acpc/*__ and __*CT/*__. Place your niftii format T1 MRI file (name it *T1_orig.nii*) in the __*acpc/*__ folder and the CT niftii file (name it *CT.nii*) in __*CT/*__. Your directory structure should now look like this: 

SUBJECTS_DIR/
    * Subj_name/
        * acpc/
            *T1_orig.nii
        * CT/
            * CT.nii

### 1. Alignment of T1 scan to AC-PC axis

*	Alignment of the T1 scan to the anterior commissure-posterior commissure axis is performed in Freeview (Figure 3). Open Freeview and load your unaligned T1 scan in the Volumes tab. To aid in axis alignment, change the cursor style “long” in Preferences  Cursor  style “Long”. You can also change the color if you wish.
*	To adjust the rotation and translation of the image, select Tools  Transform Volume. Adjust the roll (with Y (P-A)) and yaw (with Z (I-S)) as necessary to make sure the head is aligned.  Check the axial view to make sure the eyes show equally in the same slice (see Fig 3A vs. 3B, second panel for unaligned and aligned examples).  Make sure the midsagittal line is vertical in the axial view (see Fig 3A vs. 3B, first and third panels) and in the coronal view.  Choose Sample method “Cubic”.
*	Select the anterior commissure and adjust the pitch of the head so that it is in line with the posterior commissure on the horizontal axis (Fig 3A-B, last panel).
*	Finally, move to the (0, 0, 0) RAS coordinate (not TkReg RAS, just RAS).  In the Transform Volume tool, translate the image until the cursor is at the anterior commissure.   
*	Once you are satisfied that the brain is in a good orientation, click “Save Reg…” and save the transformation matrix in the acpc directory as T1_reorient.lta. Then, click “Save as…” and save the reoriented T1 file as T1.nii in the acpc directory (e.g. /usr/local/freesurfer/subjects/EC1/acpc). 

#### Include image here 

### 2. Create the freeCoG instance for the patient and run preparatory steps

In [2]:
patient = img_pipe.freeCoG(subj='EC108',
                           hem='lh',
                           subj_dir='/Applications/freesurfer/subjects',
                           fs_dir='/Applications/freesurfer/')
# you can also specify SUBJECTS_DIR and FREESURFER_HOME in your .bash_profile instead of specifying 
# in the *subj_dir* and *fs_dir* arguments.

#### 2a) **`prep_recon()`**
**`prep_recon()`** will set up the directory structure needed before running **`get_recon()`**

In [3]:
patient.prep_recon()

#### 2b) **`get_recon()`** 
**`get_recon()`** will run the Freesurfer command `recon-all`. This will take on the order of hours, depending on your machine's CPU, GPU, and whether you use parallelization.

In [14]:
patient.get_recon()

#### 2c) check pial surfaces against the MRI with **`check_pial()`**
Now, check that the pial surfaces actually correspond to the MRI.

In [4]:
#this will open a freesurfer window with the brain MRI and pial surfaces loaded
patient.check_pial()

#### 2d) create the meshes

Create the triangle-vertex mesh in .mat format with **`convert_fsmesh2mlab()`**. This will save them out to **__Meshes/EC108_lh_pial.mat__** and **__Meshes/EC108_rh_pial.mat__**. To create subcortical meshes, run **`get_subcort()`**, subcortical meshes will be in **__Meshes/subcortical/__**.

In [9]:
patient.convert_fsmesh2mlab()
#patient.get_subcort()

Making lh mesh
Making rh mesh


#### 2e) CT Registration

Register the **__CT/CT.nii__** file to **__mri/orig.nii__** file; the registered CT will be saved as **__CT/rCT.nii__**.

In [7]:
patient.reg_img('CT.nii','orig.nii')

Computing registration from /Applications/freesurfer/subjects/EC108/CT/CT.nii to /Applications/freesurfer/subjects/EC108/mri/orig.nii
Initial guess...
translation : [ 0.  0.  0.]
rotation    : [ 0.  0.  0.]
scaling     : [ 1.  1.  1.]
pre-rotation: [ 0.  0.  0.]
Optimizing using fmin_powell
translation : [-0.00255609 -0.61803397  1.        ]
rotation    : [ 0.00116414  0.00014998  0.00160618]
scaling     : [ 0.99383872  1.01005017  0.99383872]
pre-rotation: [-0.00224096  0.00174025  0.00093322]
nmi = 0.229980415331

translation : [ 0.08387597 -0.23606797  0.38196603]
rotation    : [ -5.01619537e-03   8.62185599e-04   1.79002810e-05]
scaling     : [ 0.99344068  1.01160345  0.99481757]
pre-rotation: [-0.0039123   0.0022048   0.00177942]
nmi = 0.231154842067

Optimization terminated successfully.
         Current function value: -0.231155
         Iterations: 2
         Function evaluations: 174
Saving registered CT image as /Applications/freesurfer/subjects/EC108/CT/rCT.nii


/Users/dlchang/anaconda2/lib/python2.7/site-packages/nipy/io/files.py:145: FutureWarning: Default `strict` currently False; this will change to True in a future version of nipy
  ni_img = nipy2nifti(img, data_dtype = io_dtype)


### 3. Electrode localization, anatomical labeling, and nonlinear warping

#### 3a) Marking the electrodes on the CT

Example of identification of electrode coordinates using electrode picker, opened with **`mark_electrodes()`**. 

<img src='files/ElectrodeMarker.png' style='width:700px;'>

**(A)** Demonstrates the process of picking the coordinate for the most posterior inferior grid corner. On the left, the GUI is shown with the electrode selected. The pial surface, rCT, and skull stripped MRI are displayed. The upper left shows the electrode selected in the sagittal view. The upper right shows the coronal view. The bottom right shows an axial view. The lower left displays the intensity projection map of the CT, which is useful for visualizing the entire grid. To save the coordinate, press ‘n’ to name a new device. With the center of the electrode artifact localized by the crosshairs in the axial, sagittal, and coronal views, press ‘e’ to add a point. The coordinates are automatically saved to the ‘elecs’ folder. This file can be loaded and the saved points can be plotted on the 3D surface mesh in MATLAB. This plot can be seen in the right panel. If the coordinates appear buried in the Mesh due to post operative brain shift, additional steps can be taken to project the electrode to the surface. 

**(B)** Example of identification of an electrode that is part of a subtemporal strip. The strip can be seen in the rCT.nii intensity projection map in the lower right panel. The coordinate is recorded from the center of the electrode artifact, seen in sagittal, coronal, and axial views. This coordinate can then be visualized on the 3D surface mesh in MATLAB, seen in the right panel.


In [10]:
#This will open a GUI where you can click to mark the electrodes on the registered CT 
patient.mark_electrodes()

Creating directory /Applications/freesurfer/subjects/EC108/elecs/individual_elecs
Launching electrode picker


#### 2f) Interpolate electrode grids

If you marked any grid corners, you should interpolate then project the electrodes.

In [14]:
help(patient.interp_grid)
patient.interp_grid(grid_basename='hd_grid',
                   nchans=256)

Help on method interp_grid in module img_pipe.img_pipe:

interp_grid(self, nchans=256, grid_basename='hd_grid') method of img_pipe.img_pipe.freeCoG instance
    Interpolates corners for an electrode grid
    given the four corners (in order, 1, 16, 241, 256), or for
    32 channel grid, 1, 8, 25, 32.



#### 2g) Project electrodes that are buried under the cortical surface

This can either work on interpolated grids or individual electrodes. For interpolated grids, you have the option of using the mean normal vector as the projection direction (see figure). For individual electrodes, you can specify the projection direction if you set **`use_mean_normal=False`**. You can also select whether to use the dural surface or pial surface as the surface to project to.

<img src='files/ElectrodeProjection.png' style="width: 650px;">

**A.**  The grid’s corner electrodes are manually located. We interpolate the locations of the rest of the grid electrodes using these corner coordinates, giving us the electrode grid shown in red. The green arrows are the four normal vectors calculated from the corners, and the black arrow is the mean of those normal vectors and will act as our projection direction. 

**B.** Projection of the interpolated grid (red) to the convex hull of the pial surface (blue) using the mean normal vector (black arrow). The final projected electrode grid is shown in blue.


In [19]:
help(patient.project_electrodes)

Help on method project_electrodes in module img_pipe.img_pipe:

project_electrodes(self, elecfile_prefix='hd_grid', use_mean_normal=True, surf_type='dural', num_iter=30, dilate=0.0, grid=True) method of img_pipe.img_pipe.freeCoG instance
    elecfile_prefix: prefix of the .mat file with the electrode coordinates matrix 
    use_mean_normal: whether to use mean normal vector (mean of the 4 normal vectors from the grid's 
    corner electrodes) as the projection direction
    surf_type: 'dural' or 'pial'
    projection_method: 'convex_hull','none','alphavol'
    num_iter: how many smoothing iterations when creating the dural surface
    
    Projects the electrodes of a grid based on the mean normal vector of the four grid
    corner electrodes that were manually localized from the registered CT.



In [3]:
patient.project_electrodes(elecfile_prefix='hd_grid')

Projection Params: 
	 Grid Name: hd_grid.mat 
	 Use Mean Normal: True 
	                Surface Type: dural 
	 Number of Smoothing Iterations (if using dural): 30
Normal vectors: [array([-0.97114023,  0.22398069,  0.08197134]), array([-0.95589682,  0.2857663 ,  0.06781517]), array([-0.97738423,  0.21065431,  0.01856967]), array([-0.96238506,  0.27167383,  0.00288623])]
('Projection direction vector: ', [-0.96670158371356152, 0.24801878269369992, 0.042810601502468078])
::: Loading Mesh data :::
/Applications/freesurfer/subjects/EC108/Meshes/lh_dural_trivert.mat
::: Projecting electrodes to mesh :::
[-0.96670158371356152, 0.24801878269369992, 0.042810601502468078]


/Users/dlchang/img_pipe/img_pipe/surface_warping_scripts/project_electrodes_anydirection.py:236: RuntimeWarning: invalid value encountered in greater_equal
  ok = (angleOK) & (u>=-zero) & (u<=1.0+zero) # mask
/Users/dlchang/img_pipe/img_pipe/surface_warping_scripts/project_electrodes_anydirection.py:236: RuntimeWarning: invalid value encountered in less_equal
  ok = (angleOK) & (u>=-zero) & (u<=1.0+zero) # mask
/Users/dlchang/img_pipe/img_pipe/surface_warping_scripts/project_electrodes_anydirection.py:246: RuntimeWarning: invalid value encountered in greater_equal
  ok = (ok) & (v>=-zero) & ((u+v)<=(1.0+zero))
/Users/dlchang/img_pipe/img_pipe/surface_warping_scripts/project_electrodes_anydirection.py:246: RuntimeWarning: invalid value encountered in less_equal
  ok = (ok) & (v>=-zero) & ((u+v)<=(1.0+zero))


::: Done :::
Moving /Applications/freesurfer/subjects/EC108/elecs/individual_elecs/hd_grid_orig.mat to /Applications/freesurfer/subjects/EC108/elecs/individual_elecs/preproc


#### Creating the **`*_elecs_all.mat`** file

Now we will create our **`'*_elecs_all.mat'`** montage file. This file is a `.mat` file with two structs: **`anatomy`**, which contains information about the montage labels, device origin, and anatomical labels; and **`elecmatrix`**, which contains the electrode coordinate matrix.

**`anatomy`** is structured as a __num_electrodes x 4__ matrix. The first column is the short name abbreviation of the electrode, the second column is the long name, the third column is the electrode type (i.e. depth, strip, or grid).

**`make_elecs_all()`** is an interactive function that creates the **`'*_elecs_all.mat'`** file.

In [5]:
#creates the TDT_elecs_all.mat file 
patient.make_elecs_all()

Are you adding a row that will be NaN in the elecmatrix? If not, press enter. If so, enter the number of empty rows to add: 

What is the short name prefix of the device?
AD
What is the long name prefix of the device?
AmygDepth
What is the type of the device?
depth
What is the filename of the device's electrode coordinate matrix?
amygdala_depth.mat
Finished entering devices? Enter 'y' if finished.n
Are you adding a row that will be NaN in the elecmatrix? If not, press enter. If so, enter the number of empty rows to add: 

What is the short name prefix of the device?
HD
What is the long name prefix of the device?
HdGrid
What is the type of the device?
grid
What is the filename of the device's electrode coordinate matrix?
hd_grid.mat
Finished entering devices? Enter 'y' if finished.
Are you adding a row that will be NaN in the elecmatrix? If not, press enter. If so, enter the number of empty rows to add: 

What is the short name prefix of the device?
AST
What is the long name prefix of t

In [10]:
print scipy.io.loadmat(os.path.join(patient.elecs_dir,'test_elecs_all.mat'))['eleclabels'][0:5,:]
print scipy.io.loadmat(os.path.join(patient.elecs_dir,'test_elecs_all.mat'))['elecmatrix'].shape

[[array([u'AD1'], 
      dtype='<U3')
  array([u'AmygDepth1'], 
      dtype='<U10')
  array([u'depth'], 
      dtype='<U5')]
 [array([u'AD2'], 
      dtype='<U3')
  array([u'AmygDepth2'], 
      dtype='<U10')
  array([u'depth'], 
      dtype='<U5')]
 [array([u'AD3'], 
      dtype='<U3')
  array([u'AmygDepth3'], 
      dtype='<U10')
  array([u'depth'], 
      dtype='<U5')]
 [array([u'AD4'], 
      dtype='<U3')
  array([u'AmygDepth4'], 
      dtype='<U10')
  array([u'depth'], 
      dtype='<U5')]
 [array([u'AD5'], 
      dtype='<U3')
  array([u'AmygDepth5'], 
      dtype='<U10')
  array([u'depth'], 
      dtype='<U5')]]
(272, 3)


In [16]:
#labels the electrodes and saves them to the 'anatomy' struct of the *_elecs_all.mat file
patient.label_elecs(elecfile_prefix='TDT_elecs_all')

/Applications/freesurfer/subjects
Creating labels from the freesurfer annotation file for use in automated electrode labeling
Loading electrode matrix
Loading label bankssts
Loading label caudalanteriorcingulate
Loading label caudalmiddlefrontal
Loading label cuneus
Loading label entorhinal
Loading label frontalpole
Loading label fusiform
Loading label inferiorparietal
Loading label inferiortemporal
Loading label insula
Loading label isthmuscingulate
Loading label lateraloccipital
Loading label lateralorbitofrontal
Loading label lingual
Loading label medialorbitofrontal
Loading label middletemporal
Loading label paracentral
Loading label parahippocampal
Loading label parsopercularis
Loading label parsorbitalis
Loading label parstriangularis
Loading label pericalcarine
Loading label postcentral
Loading label posteriorcingulate
Loading label precentral
Loading label precuneus
Loading label rostralanteriorcingulate
Loading label rostralmiddlefrontal
Loading label superiorfrontal
Loading l

array([[u'G1', u'L256ContactGridElectrode1', u'grid', 'inferiortemporal'],
       [u'G2', u'L256ContactGridElectrode2', u'grid', 'middletemporal'],
       [u'G3', u'L256ContactGridElectrode3', u'grid', 'middletemporal'],
       ..., 
       [u'CD8', u'CingulateDepthElectrode8', u'depth',
        'Left-Cerebral-White-Matter'],
       [u'CD9', u'CingulateDepthElectrode9', u'depth', 'ctx_lh_S_front_inf'],
       [u'CD10', u'CingulateDepthElectrode10', u'depth',
        'ctx_lh_G_front_middle']], dtype=object)

In [19]:
patient.warp_all()

### Final File Structure

SUBJECTS_DIR/
    * Subj_name/
        * CT/	
        * Meshes/	
        * acpc/	
        * ascii/	
        * cvs/
        * dicom/	
        * elecs/
            -individual_elecs/
        * label/	
        * mri/	
        * scripts/	
        * surf/	

See plotting demo.